# Assigment 4

In [33]:
import math
import numpy as np
import scipy.sparse as sp

import igl
import meshplot as mp

from math import sqrt

In [34]:
v, f = igl.read_triangle_mesh("data/irr4-cyl2.off")
tt, _ = igl.triangle_triangle_adjacency(f)

c = np.loadtxt("data/irr4-cyl2.constraints")
cf = c[:, 0].astype(np.int64)
c = c[:, 1:]
# print(cf)
# print(c)

# Tangent vector fields for scalar field design 
## Soft Constraint

In [35]:
def align_field(V, F, TT, soft_id, soft_value, llambda):
    assert(soft_id[0] > 0)
    assert(soft_id.shape[0] == soft_value.shape[0])

    
    # Edges
    e1 = V[F[:, 1], :] - V[F[:, 0], :]
    e2 = V[F[:, 2], :] - V[F[:, 0], :]

    # Compute the local reference systems for each face, T1, T2
    T1 = e1 / np.linalg.norm(e1, axis=1)[:,None]
        
    T2 =  np.cross(T1, np.cross(T1, e2))
    T2 /= np.linalg.norm(T2, axis=1)[:,None]
  
    # Arrays for the entries of the matrix
    data = []
    ii = []
    jj = []
    
    index = 0
    for f in range(F.shape[0]):
        for ei in range(3): # Loop over the edges
            
            # Look up the opposite face
            g = TT[f, ei]
            
            # If it is a boundary edge, it does not contribute to the energy
            # or avoid to count every edge twice
            if g == -1 or f > g:
                continue
                
            # Compute the complex representation of the common edge
            e  = V[F[f, (ei+1)%3], :] - V[F[f, ei], :]
            
            vef = np.array([np.dot(e, T1[f, :]), np.dot(e, T2[f, :])])
            vef /= np.linalg.norm(vef)
            ef = (vef[0] + vef[1]*1j).conjugate()
            
            veg = np.array([np.dot(e, T1[g, :]), np.dot(e, T2[g, :])])
            veg /= np.linalg.norm(veg)
            eg = (veg[0] + veg[1]*1j).conjugate()
            
            
            # Add the term conj(f)^n*ui - conj(g)^n*uj to the energy matrix
            data.append(ef);  ii.append(index); jj.append(f)
            data.append(-eg); ii.append(index); jj.append(g)

            index += 1
            
    
    sqrtl = sqrt(llambda)
    
    # Convert the constraints into the complex polynomial coefficients and add them as soft constraints
    
    # Rhs of the system
    b = np.zeros(index + soft_id.shape[0], dtype=np.complex)
    
    for ci in range(soft_id.shape[0]):
        f = soft_id[ci]
        v = soft_value[ci, :]
        
        # Project on the local frame
        c = np.dot(v, T1[f, :]) + np.dot(v, T2[f, :])*1j
        
        data.append(sqrtl); ii.append(index); jj.append(f)
        b[index] = c * sqrtl
        
        index += 1
    
    assert(b.shape[0] == index)
    
    
    # Solve the linear system
    A = sp.coo_matrix((data, (ii, jj)), shape=(index, F.shape[0])).asformat("csr")
    u = sp.linalg.spsolve(A.H @ A, A.H @ b)
    R = T1 * u.real[:,None] + T2 * u.imag[:,None]

    return R

In [36]:
def plot_mesh_field(V, F, R, constrain_faces):
    # Highlight in red the constrained faces
    col = np.ones_like(f)
    col[constrain_faces, 1:] = 0
    
    # Scaling of the representative vectors
    avg = igl.avg_edge_length(V, F)/2

    #Plot from face barycenters
    B = igl.barycenter(V, F)

    p = mp.plot(V, F, c=col)
    p.add_lines(B, B + R * avg)
    
    return p

In [37]:
R = align_field(v, f, tt, cf, c, 1e6)
plot_mesh_field(v, f, R, cf)
np.savetxt('data/softconstr_vectors.txt', R, delimiter=' ')

C:\Users\13400\AppData\Local\Temp\ipykernel_5892\1349794068.py:57: DeprecationWarning: `np.complex` is a deprecated alias for the builtin `complex`. To silence this warning, use `complex` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.complex128` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  b = np.zeros(index + soft_id.shape[0], dtype=np.complex)


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

## Hard Constraint

In [29]:
## min ||A*x||, A = [Af, xf], x = [xf, xc], xc is hard constraint
## that is, min || Af*xf + Ac*xc ||
## => Af.H*Af*Xf+Af.H*Ac*xc = 0 
def align_field_hard_constraint(V, F, TT, constr_fids, constr_vals):
    assert(constr_fids.shape[0] > 0) # I think this should be .shape[0] not just [0]
    assert(constr_fids.shape[0] == constr_vals.shape[0])

    
    # Edges
    e1 = V[F[:, 1], :] - V[F[:, 0], :]
    e2 = V[F[:, 2], :] - V[F[:, 0], :]

    # Compute the local reference systems for each face, T1, T2
    T1 = e1 / np.linalg.norm(e1, axis=1)[:,None]
        
    T2 =  np.cross(T1, np.cross(T1, e2))
    T2 /= np.linalg.norm(T2, axis=1)[:,None]
  
    # Arrays for the entries of the matrix
    data = []
    ii = []
    jj = []
    
    index = 0
    for f in range(F.shape[0]):
        for ei in range(3): # Loop over the edges
            
            # Look up the opposite face
            g = TT[f, ei]
            
            # If it is a boundary edge, it does not contribute to the energy
            # or avoid to count every edge twice
            if g == -1 or f > g:
                continue
                
            # Compute the complex representation of the common edge
            e  = V[F[f, (ei+1)%3], :] - V[F[f, ei], :]
            
            vef = np.array([np.dot(e, T1[f, :]), np.dot(e, T2[f, :])])
            vef /= np.linalg.norm(vef)
            ef = (vef[0] + vef[1]*1j).conjugate()
            
            veg = np.array([np.dot(e, T1[g, :]), np.dot(e, T2[g, :])])
            veg /= np.linalg.norm(veg)
            eg = (veg[0] + veg[1]*1j).conjugate()
            
            
            # Add the term conj(f)^n*ui - conj(g)^n*uj to the energy matrix
            data.append(ef);  ii.append(index); jj.append(f)
            data.append(-eg); ii.append(index); jj.append(g)

            index += 1
            
            
    A = sp.coo_matrix((data, (ii, jj)), shape=(index, F.shape[0])).asformat("csr")
    # split A, x according to unknown xf and constants (hard constraints) xc
    fnum = F.shape[0] # number of faces
    constr_fnum = constr_fids.shape[0] # number of constrained faces
    unconstr_fnum = fnum - constr_fnum
    unconstr_fids = []
    for fid in range(fnum):
        if (not (fid in constr_fids)):
            unconstr_fids.append(fid)
            
    # A = [Af, Ac]
    Af = A[:, unconstr_fids]
    Ac = A[:, constr_fids]
    
    # get xc -- convert constraint vectors to complex numbers on coordinates of each face
    xc = np.zeros(constr_fnum, dtype=np.complex)
    for cid in range(constr_fnum):
        fid = constr_fids[cid]
        v = constr_vals[cid] # 3d vector
        c = np.dot(v, T1[fid, :]) + np.dot(v, T2[fid, :])*1j # converted to complex number on this face
        xc[cid] = c
        
    # Solve the linear system
    xf = sp.linalg.spsolve(Af.H @ Af, -1*Af.H @ Ac @ xc)
    # convert to 3-d coordinates (unconstraint vectors)
#     print(T1.shape)
#     print(xf.real[:,None].shape)
    unconstr_vals = T1[unconstr_fids, :] * xf.real[:,None] + T2[unconstr_fids, :] * xf.imag[:,None]
    # vectors of each face
    vectors = np.zeros((fnum, 3))
    vectors[unconstr_fids] = unconstr_vals
    vectors[constr_fids] = constr_vals

    return vectors

In [32]:
vectors_hardconstr = align_field_hard_constraint(v, f, tt, cf, c)
plot_mesh_field(v, f, vectors_hardconstr, cf)
np.savetxt('data/hardconstr_vectors.txt', vectors_hardconstr, delimiter=' ')

C:\Users\13400\AppData\Local\Temp\ipykernel_5892\2741399538.py:70: DeprecationWarning: `np.complex` is a deprecated alias for the builtin `complex`. To silence this warning, use `complex` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.complex128` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  xc = np.zeros(constr_fnum, dtype=np.complex)


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

In [ ]:
#